# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [55]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [56]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [57]:
# word net installation:

# unmark if you want to use and need to install
import sys 
!{sys.executable} -m pip install wn
import wn

In [58]:
# word net import:

# unmark if you want to use:
import sys 
!{sys.executable} -m pip install hebrew_tokenizer
import hebrew_tokenizer as ht

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [59]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [60]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [61]:
# specify file names for training and test data
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
# read the data from CSV files into pandas dataframes
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [62]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [63]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [64]:
# split the training data into training and validation sets
# with a test size of 20% and shuffle=False to maintain order of the stories

X_train, X_test, y_train, y_test = train_test_split(df_train.story, df_train.gender, test_size=0.2, shuffle=False)



In [65]:
# define a pipeline for feature extraction and classification
# the pipeline consists of two steps:
# 1. TfidfVectorizer: this step converts the text data into a numerical feature matrix using TF-IDF algorithm
# 2. SGDClassifier: this step trains a linear classifier on the numerical feature matrix using SGD algorithm
clf = Pipeline([
    # step 1: TfidfVectorizer
    ('vect', TfidfVectorizer(ngram_range=(1, 2), tokenizer=lambda x: [details[1] for details in ht.tokenize(x) if details[0] == 'HEBREW' and len(details[1]) > 1])),
    # step 2: SGDClassifier
    ('clf', SGDClassifier(loss='hinge', penalty='l1', alpha=1e-5, random_state=None, tol=None))
])

# fit the pipeline on the training data
clf = clf.fit(X_train, y_train)

# make predictions on the training data
y_pred = clf.predict(df_train.story)

In [66]:
# make predictions on the test data
test_prediction = clf.predict(df_test.story)

# create a pandas dataframe from the predicted test data
df_predicted = pd.DataFrame({"test_example_id":df_test.test_example_id,"predicted_category":test_prediction})
y_test = df_train.gender


In [67]:
# calculate the macro F1 score and print the confusion matrix
# YOUR CODE HERE
print("The macro f1 score is:",f1_score(y_test, y_pred, average='macro'))
print("\nThe confusion matrix of my model is:")
print(pd.DataFrame(metrics.confusion_matrix(y_pred=y_pred, y_true=y_test), columns=['f', 'm'], index=['f', 'm']))
# create pandas dataframes to show the predicted and actual gender of the writer of the stories
# showing the first 10 and last 10 rows of the dataframes, and their shapes
pd.DataFrame({'y_test':y_test, 'y_pred':y_pred}).head(5)
pd.DataFrame({'y_test':y_test, 'y_pred':y_pred}).tail(5)
pd.DataFrame({'y_test':y_test, 'y_pred':y_pred}).shape

The macro f1 score is: 0.9531917856245944

The confusion matrix of my model is:
     f    m
f  161   17
m    8  567


,y_test,y_pred
0,m,m
1,m,m
2,f,f
3,m,m
4,f,f


,y_test,y_pred
748,m,m
749,m,m
750,m,m
751,f,m
752,m,m


(753, 2)

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [68]:
df_predicted.to_csv('classification_results.csv',index=False)